In [28]:
import torch
import torch.nn as nn

In [29]:
# CrossEntropyLoss
# 构建虚拟数据 batch_size=3即3个样本
inputs = torch.tensor([[1, 2], [1, 3], [1, 3]], dtype=torch.float)
# 设置标签、类型为长整型，第1个样本为第0类，第2、3个样本为第1类，注意类别数class是从0算起的
target = torch.tensor([0, 1, 1], dtype=torch.long)

# 通过nn.CrossEntropyLoss构造损失函数，观察3种reduction模式下Loss计算
# ------------------- CrossEntropy loss: reduction ---------------------
"""
weight:各个类别的Loss设置权值
ignore_index:忽略某个类别
reduction:计算模式可为none/sum/mean
none-逐个元素计算
sum-所有元素求和，返回标量
mean-加权平均，返回标量
"""
# def loss function # 维度上不衰减，有3个样本就有3个Loss
loss_f_none = nn.CrossEntropyLoss(weight=None, reduction='none')  
# 求和模式：把所有样本的Loss加起来
loss_f_sum = nn.CrossEntropyLoss(weight=None, reduction='sum')   
# 默认模式-均分
loss_f_mean = nn.CrossEntropyLoss(weight=None, reduction='mean')   

# forward
loss_none = loss_f_none(inputs, target)
loss_sum = loss_f_sum(inputs, target)
loss_mean = loss_f_mean(inputs, target)

# view
print("Cross Entropy Loss:\n ", loss_none, loss_sum, loss_mean)

# def loss function
# weight设置时需要注意是“向量形式”，有多少个类别就需要设置多长的向量，每个类别都需要设置weight,不想关注的类别可以设置为1.设置weight,不需要关注尺度，只需要关注各类别之间的比例。
# 设置第0类权重为1，第1类权重为2
weights = torch.tensor([1, 2], dtype=torch.float)
# weights = torch.tensor([0.7, 0.3], dtype=torch.float)

loss_f_none_w = nn.CrossEntropyLoss(weight=weights, reduction='none')
loss_f_sum = nn.CrossEntropyLoss(weight=weights, reduction='sum')
loss_f_mean = nn.CrossEntropyLoss(weight=weights, reduction='mean')

# forward
loss_none_w = loss_f_none_w(inputs, target)
loss_sum = loss_f_sum(inputs, target)
loss_mean = loss_f_mean(inputs, target)

# view
print("\nweights: ", weights)
print(loss_none_w, loss_sum, loss_mean)

Cross Entropy Loss:
  tensor([1.3133, 0.1269, 0.1269]) tensor(1.5671) tensor(0.5224)

weights:  tensor([1., 2.])
tensor([1.3133, 0.2539, 0.2539]) tensor(1.8210) tensor(0.3642)


In [30]:
# loss中都是 prediction在前，target在后
# L1Loss(), 绝对值误差

x = torch.randn(3)
y = torch.tensor([1., 1., 0.])
print(x, y)
l1loss = nn.L1Loss()
a = 0
print(l1loss(x, y))
for i in range(len(x)):
    a += abs(x[i]-y[i])
print(a/3)

tensor([1.4367, 0.3523, 1.1874]) tensor([1., 1., 0.])
tensor(0.7573)
tensor(0.7573)


In [31]:
# loss中都是 prediction在前，target在后
# MSELoss:平方损失的平均值()
x = torch.randn(3)
y = torch.tensor([1., 1., 0.])
print(x, y)
mseloss = nn.MSELoss()   # loss 是prediction和target之间的关系
print(mseloss(x, y))
a = 0
for i in range(len(x)):
    a += (y[i]-x[i])**2
print(a/3)

tensor([-0.1507, -0.3613,  0.0976]) tensor([1., 1., 0.])
tensor(1.0623)
tensor(1.0623)


In [34]:
# NLLLoss()
inputs = torch.tensor([[1, 2], [1, 3], [1, 3]], dtype=torch.float)
target = torch.tensor([0, 1, 1], dtype=torch.long)

weights = torch.tensor([1, 1], dtype=torch.float)

loss_f_none_w = nn.NLLLoss(weight=weights, reduction='none')
loss_f_sum = nn.NLLLoss(weight=weights, reduction='sum')
loss_f_mean = nn.NLLLoss(weight=weights, reduction='mean')
print(loss_f_none_w(inputs, target), 
      loss_f_sum(inputs, target), 
      loss_f_mean(inputs, target))

tensor([-1., -3., -3.]) tensor(-7.) tensor(-2.3333)


In [26]:
# BCELoss():二分类，多分类都可以，其实就是多分类！！！输出在[0, 1]
# 虚拟输入：二分类所以有2个神经元，4个样本，第1个样本：第1个神经元输出值为1，第2个神经元输出值为2
inputs = torch.tensor([[1, 2], [2, 2], [3, 4], [4, 5]], dtype=torch.float)
# BCE与CrossEntropyLoss的标签不同，是浮点类数据，每个神经元一一对应的计算Loss,而非一整个神经元向量计算Loss
target = torch.tensor([[1, 0], [1, 0], [0, 1], [0, 1]], dtype=torch.float)

target_bce = target


weights = torch.tensor([1, 1], dtype=torch.float)

loss_f_none_w = nn.BCELoss(weight=weights, reduction='none')
loss_f_sum = nn.BCELoss(weight=weights, reduction='sum')
loss_f_mean = nn.BCELoss(weight=weights, reduction='mean')

# forward
loss_none_w = loss_f_none_w(inputs, target_bce)
loss_sum = loss_f_sum(inputs, target_bce)
loss_mean = loss_f_mean(inputs, target_bce)

RuntimeError: all elements of input should be between 0 and 1

In [ ]:
# 输入值并没有在[0,1]之间，由于BCE二分类交叉熵损失函数衡量的是两个概率分布之间的差异，因此输入应当在[0,1]之间。
# 因此，需要将输入值经过激活函数 sigmoid()进行压缩，使其输出值在(0,1)之间。